In [16]:
import pandas as pd
import numpy as np

In [17]:
esg_tickers = pd.read_csv("../Refinitiv ESG Final Data for Analysis.csv")["Symbol"]

company_data = pd.read_csv("../company_data.csv")

# tickers for which we have financial data
tickers = company_data[company_data['days_since_ipo'] > 180]['ticker']

# use mask to filter out tickers found in esg dataset that do not exist and dual classes of stock
# PEAK,
# PXD (pioneer energy, aquired by exxon mobil),
# WRK (sidney australia listing)
# CDAY renamed to DAY
# FLT listed in australia
# GOOG (class C to GOOGL)
# FOX (class B to FOXA)
# NWS (class B to NWSA)

esg_tickers = esg_tickers[~esg_tickers.isin(["PEAK", "PXD", "WRK", "CDAY", "FLT", "GOOG", "FOX", "NWS"])]

# intersection of tickers in ESG data and the SP500 data we have
# todo: only get the timeseries data for the ESG tickers we have
tickers = pd.Series(list(set(esg_tickers) & set(tickers)), name = 'ticker').sort_values(ignore_index=True)

# save for other files to use
#tickers.to_csv("../tickers_to_keep.csv", index=False)

In [43]:
timeseries_13_18 = pd.read_csv("SP500_raw_timeseries_1-1-13--12-31-18.csv")
timeseries_19_24 = pd.read_csv("SP500_raw_timeseries_1-1-19--11-1-24.csv")

# remove columns (days) with no data, for example holidays
timeseries_13_18 = timeseries_13_18.dropna(axis=1, how='all')
timeseries_19_24 = timeseries_19_24.dropna(axis=1, how='all')

# transpose so each column turns in a timeseries for one ticker
# also reverse order so dates increase from top to bottom
timeseries_13_18 = timeseries_13_18.set_index('ticker').T.iloc[::-1]
timeseries_13_18.columns.name = None
timeseries_19_24 = timeseries_19_24.set_index('ticker').T.iloc[::-1]
timeseries_19_24.columns.name = None

# Concatenate along the rows (axis=0)
timeseries = pd.concat([timeseries_13_18.reset_index(), timeseries_19_24.reset_index()],
                       ignore_index=True, sort=False)

timeseries.rename(columns={timeseries.columns[0]: 'date'}, inplace=True)
timeseries.set_index('date', inplace=True)
timeseries.index = pd.to_datetime(timeseries.index)

# only keep tickers we're interested in
timeseries = timeseries.filter(items=tickers)

In [19]:
spy_timeseries = pd.read_csv("spy_raw_timeseries_13-24.csv").dropna(axis = 1, how='all')

# transpose and reverse order so dates increase from top to bottom
spy_timeseries = spy_timeseries.T.iloc[:0:-1]

spy_timeseries.rename(columns={spy_timeseries.columns[0]: 'SPY'}, inplace=True)

#print(spy_timeseries)
spy_timeseries = (spy_timeseries/spy_timeseries.shift(1)).iloc[1:].map(np.log)

spy_mean_log_return = spy_timeseries.mean() * 252
spy_annual_volatility = spy_timeseries.std() * np.sqrt(252)

#spy_timeseries.to_csv('spy_timeseries_13-24.csv')

#print(spy_mean_log_return, spy_annual_volatility)

In [44]:
# replace each entry with the log return compared to the previous day
# first row will turn into NaN so remove it
timeseries = (timeseries/timeseries.shift(1)).iloc[1:].map(np.log)

In [45]:
#calculate performance averages for each ticker
# mean log return is used in markowitz optimization
# this volatility calculation is for information only

#remember that each ticker is a column with dates increasing from top to bottom
performance_summaries = pd.DataFrame(timeseries.mean(axis = 0) * 252, columns = ['mean_log_return'])

performance_summaries['standard_deviation'] = timeseries.std(axis = 0) * np.sqrt(252)

performance_summaries.rename_axis('ticker')

#remember to write to csv as transpose so that tickers are easily filtered columns
print(performance_summaries.T.head())

                           A      AAPL      ABBV      ABNB       ABT  \
mean_log_return     0.128544  0.205628  0.148640 -0.015110  0.110685   
standard_deviation  0.259354  0.281245  0.264293  0.493811  0.228303   

                        ACGL       ACN      ADBE       ADI       ADM  ...  \
mean_log_return     0.158220  0.136236  0.214281  0.138227  0.054555  ...   
standard_deviation  0.249982  0.238814  0.318616  0.297966  0.261646  ...   

                         WTW        WY      WYNN       XEL       XOM  \
mean_log_return     0.100943  0.005695 -0.017787  0.075307  0.021920   
standard_deviation  0.224483  0.303562  0.465526  0.205717  0.262946   

                         XYL       YUM       ZBH      ZBRA       ZTS  
mean_log_return     0.124249  0.085279  0.041511  0.189462  0.150742  
standard_deviation  0.264647  0.235212  0.259949  0.378174  0.254815  

[2 rows x 478 columns]


In [46]:
# calculate covariance of all tickers
# use cov_matrix to calculate correlation between all tickers
# remove tickers that are too highly correlated

# for every ticker, figure out the first day present in the dataset
# then cov(A, B) and corr(A,B) only use the data that is present for both A and B

first_valid_dates = [timeseries[ticker].first_valid_index()
                       for ticker in tickers]

In [56]:
def weight_returns(r):
    
    if r <= 0:
        return r * 1.1
    
    return r * 0.9

def covariances(x, y):
    
    n = len(x)

    x_bar = np.mean(x)
    y_bar = np.mean(y)    
    
    raw_covariance = np.sum((x-x_bar) * (y-y_bar)) / (n-1)
    
    
    # weight losers more heavily than winners
    x_prime = np.where(x <= 0, x * 1.1, x * 0.9)
    y_prime = np.where(y <= 0, y * 1.1, y * 0.9)
    
    x_prime_bar = np.mean(x_prime)
    y_prime_bar = np.mean(y_prime)
    
    weighted_covariance = np.sum((x_prime - x_prime_bar) * 
                                 (y_prime - y_prime_bar)) / (n-1)
    #print(raw_covariance, weighted_covariance, weighted_covariance/raw_covariance)
    return raw_covariance, weighted_covariance
    

# calculate covariance of all tickers
raw_cov_matrix = pd.DataFrame(np.nan, index=tickers, columns=tickers)
adjusted_cov_matrix = pd.DataFrame(np.nan, index=tickers, columns=tickers)

for i in range(len(tickers)):
    #print(f"calculating covariances for {tickers[i]}         ", end="\r", flush=True)
    for j in range(i, len(tickers)):  # Loop over upper triangle

        # start at the later date after which both tickers have data
        start_date = pd.to_datetime(max(first_valid_dates[i], first_valid_dates[j]))
        
        ticker_i_slice = timeseries.loc[timeseries.index >= start_date, tickers[i]]
        ticker_j_slice = timeseries.loc[timeseries.index >= start_date, tickers[j]]
        
        # calculate and write both raw and adjusted (weighted) covariances
        raw, adj = covariances(ticker_i_slice, ticker_j_slice)
        
        raw_cov_matrix.iloc[i, j] = raw_cov_matrix.iloc[j, i] = raw
        adjusted_cov_matrix.iloc[i, j] = adjusted_cov_matrix.iloc[j, i] = adj


In [58]:
# use cov_matrix to calculate corr_matrix

std_devs = np.sqrt(np.diagonal(adjusted_cov_matrix.values))
# Create a matrix of standard deviations (outer product of std_devs with itself)
# Compute correlation matrix
corr_matrix = adjusted_cov_matrix / np.outer(std_devs, std_devs)
'''
# Apply upper triangle mask to get the upper triangle values
# k=1 to exclude the diagonal
upper_triangle = (corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1) == 1)).stack()

high_correlations = [(index[0], index[1], value) 
                     for index, value in upper_triangle.items() 
                     if value > 0.95]

print(high_correlations)

# the highly correlated tickers are mostly different classes of the same ticker
# [('FRT', 'REG', 0.9118897518213701), ('MET', 'PRU', 0.9005313958213659)]
# significantly different enough over 10 years to leave both in.
'''

"\n# Apply upper triangle mask to get the upper triangle values\n# k=1 to exclude the diagonal\nupper_triangle = (corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1) == 1)).stack()\n\nhigh_correlations = [(index[0], index[1], value) \n                     for index, value in upper_triangle.items() \n                     if value > 0.95]\n\nprint(high_correlations)\n\n# the highly correlated tickers are mostly different classes of the same ticker\n# [('FRT', 'REG', 0.9118897518213701), ('MET', 'PRU', 0.9005313958213659)]\n# significantly different enough over 10 years to leave both in.\n"

In [57]:
nan_count = adjusted_cov_matrix.isna().sum().sum()

print(nan_count)

#print(adjusted_cov_matrix.head())
#print(raw_cov_matrix.head())

0


In [73]:
n = len(tickers)

# diagonal matrix of volatilities
vol_matrix = np.diag(np.diag(adjusted_cov_matrix.values))

# matrix of average correlations, except for diagonal of 1s
rho = np.mean(corr_matrix)
average_corr_matrix = np.full((n, n), rho)
np.fill_diagonal(average_corr_matrix, 1)

# F = V C V
structured_cov_matrix = vol_matrix @ average_corr_matrix @ vol_matrix

#print(f"vol_matrix positive semi definite: {np.all(np.linalg.eigvals(vol_matrix) > 0)}")
#print(f"average_corr_matrix positive semi definite: {np.all(np.linalg.eigvals(average_corr_matrix) > 0)}")
#print(f"structured_cov_matrix positive semi definite: {np.all(np.linalg.eigvals(structured_cov_matrix) > 0)}")


print("Condition number of sample covariance matrix:", np.linalg.cond(raw_cov_matrix))
print("Condition number of adjusted covariance matrix:", np.linalg.cond(adjusted_cov_matrix))

print(f"sample cov matrix positive semi definite: {np.all(np.linalg.eigvals(raw_cov_matrix) > 0)}")
print(f"adjusted cov matrix positive semi definite: {np.all(np.linalg.eigvals(adjusted_cov_matrix) > 0)}")

delta = 0.5

lw_adjusted_cov_matrix = (1-delta) * adjusted_cov_matrix + delta * structured_cov_matrix

eigenvalues, eigenvectors = np.linalg.eigh(lw_adjusted_cov_matrix)

# Clip eigenvalues smaller than epsilon
eigenvalues = np.clip(eigenvalues, 1e-6, None)
lw_adjusted_cov_matrix = eigenvectors @ np.diag(eigenvalues) @ eigenvectors.T

print(f"determinant {np.linalg.det(lw_adjusted_cov_matrix)}")

print("lw adjusted cov matrix positive semidefinite?", np.all(eigenvalues >= 0))

print("Condition number of lw adjusted cov matrix:", np.linalg.cond(lw_adjusted_cov_matrix))

Condition number of sample covariance matrix: 85650.87924068997
Condition number of adjusted covariance matrix: 67175.32044747974
sample cov matrix positive semi definite: False
adjusted cov matrix positive semi definite: False
determinant 0.0
lw adjusted cov matrix positive semidefinite? True
Condition number of lw adjusted cov matrix: 33792.0771114569


In [74]:
#write dataframes to csv

adjusted_cov_matrix_df = pd.DataFrame(lw_adjusted_cov_matrix, columns=tickers)
adjusted_cov_matrix_df.set_index(tickers, inplace=True)
adjusted_cov_matrix_df.rename_axis('ticker', inplace=True)

adjusted_cov_matrix_df.to_csv("sp500_adjusted_cov_matrix.csv", index=True)

In [75]:
raw_cov_matrix.set_index(tickers, inplace=True)
raw_cov_matrix.rename_axis('ticker', inplace=True)

print(raw_cov_matrix.head())
raw_cov_matrix.to_csv("sp500_raw_cov_matrix.csv", index = True)

ticker         A      AAPL      ABBV      ABNB       ABT      ACGL       ACN  \
ticker                                                                         
A       0.000267  0.000127  0.000099  0.000176  0.000132  0.000095  0.000135   
AAPL    0.000127  0.000314  0.000074  0.000218  0.000105  0.000088  0.000134   
ABBV    0.000099  0.000074  0.000277  0.000004  0.000105  0.000073  0.000084   
ABNB    0.000176  0.000218  0.000004  0.000968  0.000087  0.000070  0.000174   
ABT     0.000132  0.000105  0.000105  0.000087  0.000207  0.000084  0.000112   

ticker      ADBE       ADI       ADM  ...       WTW        WY      WYNN  \
ticker                                ...                                 
A       0.000159  0.000163  0.000096  ...  0.000096  0.000146  0.000165   
AAPL    0.000188  0.000178  0.000081  ...  0.000090  0.000140  0.000170   
ABBV    0.000096  0.000090  0.000075  ...  0.000070  0.000096  0.000110   
ABNB    0.000273  0.000270  0.000069  ...  0.000096  0.000168  0

In [ ]:
# print(timeseries.shape)
timeseries.to_csv("sp500_timeseries_13-24.csv", index=True)

In [8]:

performance_summaries.T.to_csv("sp500_performance_summaries.csv", index = True)